# MATH2504 Project 2 2022 Semester 2 Submission

[Assignment Instructions](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/project2.html)

Student names: Limao Chang, Tiarne Graves

In [1]:
include("simulation_script.jl")
include("test/scenarios.jl")

## Project Structure

In the root folder is `simulation_script.jl`, which imports and loads the `GeneralizedJacksonSim` module, as well as some other packages that are useful when working with `GeneralizedJacksonSim`.

The `src/` folder contains the source code for `GeneralizedJacksonSim`.

The `test/` folder contains tests which can be run in `test/runtests.jl`, as well as the four provided test scenarios, which can be found in `test/scenarios.jl`.

## Perspective Seminar

## Task 1

**Features of amusement park that are not captured by model**:
- Amusement park attractions usually serve multiple people simultaneously, whereas the model assumes jobs can only be serviced one at a time
- People may cut in line and skip queues, so in reality it is not really a queue in the first-in, first-out (FIFO) sense

**Model assumptions that are unrealistic for the application of a theme park**:
- Assuming that queues (and by extension the theme park) have infinite capacity
- Assuming that people will unconditionally stay in a queue until they get to the front of the line - they may leave because the queue is too long, they want to go to another attraction, etc.
- Assuming that the time taken to move between attractions or leave the theme park is instant, as travelling between attractions takes time
- Service duration is independent from all other services - if people are at the theme park together as a party (e.g. a family travelling around the theme park), their service durations will not be independent from each other
- Arrival of jobs to nodes is exponentially distributed - the arrival rate will likely be higher during the day (more traffic) than in the evening, for example.

**If you were actually using this model to assess congestion at the park, would it be useful or not?**

## Task 2

The total steady state mean queue lengths are plotted against $\rho$* below for the four scenarios.

In [ ]:
"""
    plot_total_ss_mean_queue_length(net::NetworkParameters)

Plots total steady state mean queue length as a function of ρ* for the given scenario.
"""
function plot_total_ss_mean_queue_length(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.1:0.01:0.9
    total_ss_mean_queue_lengths = zeros(length(ρ_star_values))

    for (index, ρ_star) in enumerate(ρ_star_values)
        # adjust network parameters
        adjusted_scenario = set_scenario(net, ρ_star)
        ρ = compute_ρ(adjusted_scenario)

        total_ss_mean_queue_lengths[index] = sum(ρ ./ (1 .- ρ))
    end

    return plot(ρ_star_values,
                total_ss_mean_queue_lengths,
                xlabel="ρ",
                ylabel="Total Steady State Mean Queue Length",
                title="Scenario $scenario_number Theoretical")
end

p1 = plot_total_ss_mean_queue_length(scenario1, 1)
p2 = plot_total_ss_mean_queue_length(scenario2, 2)
p3 = plot_total_ss_mean_queue_length(scenario3, 3)
p4 = plot_total_ss_mean_queue_length(scenario4, 4)
        
plot(p1, p2, p3, p4, layout=(2, 2), legend=false, size=(800, 800))

## Task 3

**Simulation engine**: `src/GeneralizedJacksonSim.jl` contains the `GeneralizedJacksonSim` module, as well as the simulation engine used (`sim_net()` below).

```julia
# src/GeneralizedJacksonSim.jl
"""
A discrete event simulation engine for Open Generalized Jackson Networks.
"""
module GeneralizedJacksonSim

import Base: isless
using Accessors, DataStructures, Distributions, StatsBase, Parameters, LinearAlgebra,
    Random, Plots

include("network_parameters.jl")
include("state.jl")
include("event.jl")

export NetworkParameters, compute_ρ, maximal_alpha_scaling, set_scenario, sim_net

"""
Runs a discrete event simulation of an Open Generalized Jackson Network `net`.

The simulation runs from time `0` to `max_time`.

Statistics about the total mean queue lengths are recorded from `warm_up_time` onwards
and the estimated value is returned.

This simulation does NOT keep individual customers' state, it only keeps the state which is
the number of items in each of the nodes.
"""
function sim_net(net::NetworkParameters;
                 max_time=10^6, warm_up_time=10^4, seed::Int64=42)::Float64
    Random.seed!(seed)

    # create priority queue and add standard events
    priority_queue = BinaryMinHeap{TimedEvent}()
    for q in 1:net.L
        push!(priority_queue, TimedEvent(ExternalArrivalEvent(q), 0.0))
    end
    push!(priority_queue, TimedEvent(EndSimEvent(), max_time))

    # initialise state and time
    state = QueueNetworkState(zeros(Int64, net.L), net.L, net)
    time = 0.0

    # set up queues integral for computing total mean queue length
    queues_integral = zeros(net.L)
    last_time = 0.0

    """
    Records the queue integral of the given state at the given point in time.
    """
    function record_integral(time::Float64, state::State)
        (time >= warm_up_time) && (queues_integral += state.queues * (time - last_time))
        last_time = time
    end

    record_integral(time, state)

    # simulation loop
    while true
        # process the next upcoming event
        timed_event = pop!(priority_queue)
        time = timed_event.time
        new_timed_events = process_event(time, state, timed_event.event)

        isa(timed_event.event, EndSimEvent) && break

        # add new spawned events to queue
        for nte in new_timed_events
            push!(priority_queue, nte)
        end

        # record mean queue length
        record_integral(time, state)
    end

    return sum(queues_integral / max_time)
end

end  # end of module
```


The other functionalities of the simulation engine (`NetworkParameters`, `State`s, `Event`s, and the functions related to each of them) have been separated into their own files, to keep the code nice and structured. Below are some snippets from each file:

```julia
# src/network_parameters.jl
"""
    NetworkParameters

# Fields
- `L::Int`: the dimension of the network (number of nodes)
- `α_vector::Vector{Float64}`: the external arrival rates α_i >= 0
- `μ_vector::Vector{Float64}`: the service rates μ_i > 0
- `P::Matrix{Float64}`: the L×L routing matrix P
- `c_s::Float64`: squared coefficient of variation of the service processes, defaults to 1.0
"""
@with_kw struct NetworkParameters
    L::Int
    α_vector::Vector{Float64}
    μ_vector::Vector{Float64}
    P::Matrix{Float64}
    c_s::Float64 = 1.0
end

# src/state.jl
"""
    QueueNetworkState

# Fields
- `queues::Vector{Int}`: vector of number of customers in each queue
- `num_queues::Int`: number of queues, equivalent to `length(queues)`
- `net::NetworkParameters`: the parameters of the network
"""
mutable struct QueueNetworkState <: State
    queues::Vector{Int}
    num_queues::Int
    net::NetworkParameters
end

"""
    next_arrival_time(s::State, q::Int)

Generates the next arrival time for the `q`th server. The duration of time between
external arrivals is exponentially distributed with mean 1 / s.net.α_vector[q].
"""
next_arrival_time(s::State, q::Int) = rand(Exponential(1/s.net.α_vector[q]))

"""
    next_service_time(s::State, q::Int)

Generates the next service time for the `q`th server. The service duration is gamma
distributed.
"""
next_service_time(s::State, q::Int) = rand(rate_scv_gamma(s.net.μ_vector[q], s.net.c_s))

# src/event.jl
"""
    process_event(time::Float64, state::State, event::ExternalArrivalEvent)

Process an arrival event from outside the system, and spawns a list of events that occur as
a consequence of this arrival.

On arrival, if the server is free (no jobs in the buffer/queue), the job starts to receive
service. If the server is busy, the job queues for service and waits for its turn.

The time between external arrival events for a given server is exponentially distributed,
and the service duration is gamma distributed.
"""
function process_event(time::Float64, state::State, event::ExternalArrivalEvent)
    q = event.q
    state.queues[q] += 1  # add to queue
    new_timed_events = TimedEvent[]

    # prepare next external arrival for this particular server
    push!(new_timed_events,
        TimedEvent(ExternalArrivalEvent(q), time + next_arrival_time(state, q)))

    # start serving this job if it is the only one in the queue
    if state.queues[q] == 1
        push!(new_timed_events,
            TimedEvent(EndOfServiceEvent(q), time + next_service_time(state, q)))
    end
    return new_timed_events
end

"""
    process_event(time::Float64, state::State, event::EndOfServiceEvent)

Process an end-of-service event, and spawns a list of events that occur as a consequence of
this end of service.

When a job completes service at a buffer, it either leaves the system, or moves to another
buffer (both happen immediately). After completing service in server i, a job moves to
server j with probability P[i, j], where P is the routing matrix.
"""
function process_event(time::Float64, state::State, event::EndOfServiceEvent)
    q = event.q
    state.queues[q] -= 1  # remove from queue
    @assert state.queues[q] >= 0
    new_timed_events = TimedEvent[]

    # if there is another customer in the queue, start serving them
    if state.queues[q] > 0
        service_time = next_service_time(state, q)
        push!(new_timed_events, TimedEvent(EndOfServiceEvent(q), time + service_time))
    end

    # simulate the next location for this job; indices 1:L are the probabilities of moving
    # to another server in the system, and the last index is the probability of exiting
    # the system
    L = state.net.L
    next_loc_weights = state.net.P[q, :]
    push!(next_loc_weights, 1 - sum(next_loc_weights))
    @assert sum(next_loc_weights) == 1
    next_loc = sample(1:L+1, Weights(next_loc_weights))

    if next_loc <= L
        # job is staying in the system
        state.queues[next_loc] += 1

        # start serving job if it is the only one in the queue
        if state.queues[next_loc] == 1
            service_time = next_service_time(state, next_loc)
            push!(new_timed_events,
                TimedEvent(EndOfServiceEvent(next_loc), time + service_time))
        end
    end
    return new_timed_events
end
```

**Test 1**: the code for test 1 can be found in `test/task3_test1.jl`. We decided to advantage of multi-threading for this test, since each simulation can be run independently, and their results can be stored without affecting any other simulations (see the source code for more details). To make use of multi-threading, you have to run Julia with the `-t` or `--threads` option, and specify the number of threads you want to utilise, e.g. `julia -t 8` to run Julia with 8 threads. The test can then by executed using:
```julia
task3_test1(
    [scenario1, scenario2, scenario3, scenario4],  # vector of all scenarios to test
    verbose=false,  # whether to print out progress messages as the simulations are run
    multithreaded=true  # whether to utilise multi-threading (julia needs to be run with
                        # `julia -t n` where n is the number of threads)
)
```

An example output is given below, for `max_time` ranging from 10^3 to 10^5 (beyond 10^5 is unreasonable for scenario 4 - it took around half an hour to run scenario 4 with `max_time = 10^6`). In every scenario, the absolute relative errors seem to decrease as $\rho^*$ gets larger, and also as `max_time` gets larger.

<center>
    <img src="img/task3test1.png" width="640" />
</center>

## Task 4

## Task 5